# Last week Issue
-------------------------------------------------------------------

### Feature Engineering
---------------------------------------------------------------------
- n-gram
- K-fold
- 형용사,부사,동사
- 긍정/부정 단어

In [1]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
from sklearn import cross_validation
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

with open("C:\Workspace\\Project\\BI_text_analysis\\data\\reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        pass
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)
    
import random
random_data = random.sample(review_data,len(review_data))

reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"].lower())

y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
y_data=np.array(y_data)

C:\Users\samsung\Miniconda3\envs\workspace\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## n-gram & K-fold

In [2]:
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1,2), lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(reviewText)

skf = cross_validation.StratifiedKFold(y_data, n_folds=10,random_state=500)
for train_index, test_index in skf:
    X_train, X_test = D[train_index], D[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

result = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
result.predict(X_test)
print("SVM :",accuracy_score(result.predict(X_test),y_test))

logreg = linear_model.LogisticRegression()
logreg.fit(X_train, y_train)
logreg.predict(X_test) == y_test

print("Logistic Regression :",accuracy_score(logreg.predict(X_test), y_test))

SVM : 0.632782310004
Logistic Regression : 0.617539716617


## 형용사,부사,동사 추출

In [3]:
def pos_tagging(document):
    result = nltk.pos_tag( nltk.word_tokenize(document))    
    last_result = ['/'.join(t) for t in result]
    return last_result

tag = [pos_tagging(i) for i in reviewText]
tagging_VB_RB_JJ = []
for i in range(len(reviewText[:1000])):
    tagging = []
    for j in tag[i]:
        if j.split("/")[1] == "JJ" or j.split("/")[1] == "JJR" or j.split("/")[1] == "JJS" or j.split("/")[1] == "RB" or j.split("/")[1] == "RBR" or j.split("/")[1] == "RBS" or j.split("/")[1] == "VB" or j.split("/")[1] == "VBD" or j.split("/")[1] == "VBG" or j.split("/")[1] == "VBN" or j.split("/")[1] == "VBP" or j.split("/")[1] == "VBZ":
            tagging.append(j.split("/")[0])
    join_each_tag = " ".join(tagging)
    tagging_VB_RB_JJ.append(join_each_tag)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1,2), lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(tagging_VB_RB_J)

result = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
result.predict(X_test)
print("SVM :",accuracy_score(result.predict(X_test),y_test))

logreg = linear_model.LogisticRegression()
logreg.fit(X_train, y_train)
logreg.predict(X_test) == y_test

print("Logistic Regression :",accuracy_score(logreg.predict(X_test), y_test))



0
100
200
300
400
500
600
700
800
900
SVM : 0.632782310004
Logistic Regression : 0.617539716617


## 긍정/부정 단어

In [4]:
# readline_all.py
neg=[]
f = open("C:\Workspace\\Project\\BI_text_analysis\\data\\neg.txt", 'r')
while True:
    line = f.readline()
    if not line: break
    neg.append(line)
f.close()

# readline_all.py
pos=[]
f = open("C:\Workspace\\Project\\BI_text_analysis\\data\\pos.txt", 'r')
while True:
    line = f.readline()
    if not line: break
    pos.append(line)
f.close()

def delete_blank(document):
    result=[]
    for i in range(len(document)):
        if i % 2 == 0:
            result.append(document[i])
    return result

pos_Text = delete_blank(pos)
neg_Text = delete_blank(neg)

vailidation_Text = pos_Text+neg_Text

vailidation_result = []
for i in vailidation_Text:
    a = i.split("\n")
    vailidation_result.append(a[0])

filtered_sentence_result = []
join_text = []


for i in reviewText:
    word_tokens = word_tokenize(i)
    filtered_sentence = [w for w in word_tokens if w in vailidation_result]
    filtered_sentence_result.append(filtered_sentence)

join_filtered_sentence_result = []

for i in filtered_sentence_result:
    a = " ".join(i)
    join_filtered_sentence_result.append(a)


vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1,2), lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(join_filtered_sentence_result)

y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
y_data=np.array(y_data)



skf = cross_validation.StratifiedKFold(y_data, n_folds=10,random_state=500)
for train_index, test_index in skf:
    X_train, X_test = D[train_index], D[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]



result = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
result.predict(X_test)
print("SVM :",accuracy_score(result.predict(X_test),y_test))

logreg = linear_model.LogisticRegression()
logreg.fit(X_train, y_train)
logreg.predict(X_test) == y_test

print("Logistic Regression :",accuracy_score(logreg.predict(X_test), y_test))

SVM : 0.632782310004
Logistic Regression : 0.617539716617


### Model 선택
----------------------------------------------------------
- SVM
- Logistic regression

# This week Issue
----------------------------------------
### Variables
--------------------------------------------
- Max_df, Min_df ratio
- Test set ratio

###  7 Techniques to Handle Imbalanced Data(Resample the training set)
-----------------------------------------------------------------
- Bagging
- RandomForest
- ExtraTrees
- AdaBoost
- GradientBoosting

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1,2), lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(reviewText)
skf = cross_validation.StratifiedKFold(y_data, n_folds=10,random_state=500)
for train_index, test_index in skf:
    X_train, X_test = D[train_index], D[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

## Bagging

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
seed = 7
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train)
result.predict(X_test)
print("Bagging :",accuracy_score(result.predict(X_test),y_test))

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
seed = 7
num_trees = 100
max_features = 3
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X_train, y_train)
result.predict(X_test)
print("RandomForest :",accuracy_score(result.predict(X_test),y_test))

## ExtraTrees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

seed = 7
num_trees = 100
max_features = 7
model = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X_train, y_train)
result.predict(X_test)
print("ExtraTrees :",accuracy_score(result.predict(X_test),y_test))

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 30
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train)
result.predict(X_test)
print("AdaBoost :",accuracy_score(result.predict(X_test),y_test))

## GradientBoosting

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
seed = 7
num_trees = 100
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train)
result.predict(X_test)
print("GradientBoosting :",accuracy_score(result.predict(X_test),y_test))